# PyTorch에서 다른 모델의 매개변수를 사용하여 빠르게 모델 시작하기(warmstart)

모델을 부분적으로 불러오거나, 혹은 부분적인 모델을 불러오는 것은 학습 전이(transfer learning)나 복잡한 모델을 새로 학습할 때 자주 접하는 시나리오 이다.

학습 매개변수를 활용하면 학습 과정을 빠르게 시작할 수 있으며, 그러면 모델을 처음 훈련시킬 때보다 빠르게 수렴할 수 있다.

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [4]:
import torch.nn.functional as F

class NetA(nn.Module):
  def __init__(self):
    super(NetA, self).__init__()
    
    self.conv1 = nn.Conv2d(3, 6, 5)
    self.conv2 = nn.Conv2d(6, 16, 5)
    
    self.pool = nn.MaxPool2d(2, 2)
    
    self.fc1 = nn.Linear(16 * 5 * 5, 120)
    self.fc2 = nn.Linear(120, 84)
    self.fc3 = nn.Linear(84, 10)
    
  def forward(self, x):
    x = self.pool(F.relu(self.conv1(x)))
    x = self.pool(F.relu(self.conv2(x)))
    x = x.view(-1, 16 * 5 * 5)
    x = F.relu(self.fc1(x))
    x = F.relu(self.fc2(x))
    x = self.fc3(x)
    return x
  
class NetB(nn.Module):
  def __init__(self):
    super(NetB, self).__init__()
    
    self.conv1 = nn.Conv2d(3, 6, 5)
    self.conv2 = nn.Conv2d(6, 16, 5)
    
    self.pool = nn.MaxPool2d(2, 2)
    
    self.fc1 = nn.Linear(16 * 5 * 5, 120)
    self.fc2 = nn.Linear(120, 84)
    self.fc3 = nn.Linear(84, 10)
    
  def forward(self, x):
    x = self.pool(F.relu(self.conv1(x)))
    x = self.pool(F.relu(self.conv2(x)))
    x = x.view(-1, 16 * 5 * 5)
    x = F.relu(self.fc1(x))
    x = F.relu(self.fc2(x))
    x = self.fc3(x)
    return x
  
netA = NetA()
netB = NetB()

In [5]:
PATH = 'model.pt'

torch.save(netA.state_dict(), PATH)

## 모델 B 로 불러오기


In [6]:
netB.load_state_dict(torch.load(PATH))

<All keys matched successfully>

In [7]:
netB.eval()

NetB(
  (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)